In [1]:
# 预训练模型换pubmedber
#加个k_fold OK!
# 把border（Threshold）调低一点试试 
#将nan填充值由“NotANumber”变为“0” OK!
#将模型由BertForSequenceClassification换成XLNetForSequenceClassification
#阈值参与到了模型训练和预测中                                  #####
#那么它是如何发挥作用的，为何在预测步骤调整阈值反而会让LB上升？####
#在最前面就将阈值调成0.02试试
#使用结合了abstract和text的pubmedbert ok
#弄清楚logger是怎么发挥作用的

In [2]:
import os
import sys
import pandas as pd
import numpy as np

In [4]:
import math
import random
import time
import warnings

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import transformers as T
from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

In [5]:
DATA_DIR = "./"
OUTPUT_DIR = "./"

In [6]:
warnings.filterwarnings("ignore")

In [7]:
device = torch.device("cuda")

In [8]:
def init_logger(log_file=OUTPUT_DIR + "train.log"):
    from logging import INFO, FileHandler, Formatter, StreamHandler, getLogger

    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()

In [9]:
#Set the seed value all over to make the result reproducible
def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed = 42
seed_torch(seed)

In [10]:
train = pd.read_csv(r"C:\Users\dai\data\paper_classification\train.csv")
test = pd.read_csv(r"C:\Users\dai\data\paper_classification\test.csv")

#拿sample当submission的样板
sub = pd.read_csv(r"C:\Users\dai\data\paper_classification\sample_submit.csv", header=None)
sub.columns = ["id", "judgement"]

In [11]:
#填充nan
train['abstract'].fillna(value='0', inplace = True)
test['abstract'].fillna(value='0', inplace = True)


#融合title + abstract
train["text"] = train["title"] + train["abstract"]
train = train.drop(['abstract', 'title'], axis=1)
test["text"] = test["title"] + test["abstract"]
test = test.drop(['abstract', 'title'], axis=1)

In [12]:
test

,id,text
0,27145,Estimating the potential effects of COVID-19 p...
1,27146,Leukoerythroblastic reaction in a patient with...
2,27147,[15O]-water PET and intraoperative brain mappi...
3,27148,Adaptive image segmentation for robust measure...
4,27149,Comparison of Epidemiological Variations in CO...
...,...,...
40829,67974,"Knowledge, Attitude, and Practices of Healthca..."
40830,67975,Safety and Efficacy of Anti-Il6-Receptor Tocil...
40831,67976,Functional imaging of head and neck tumors usi...
40832,67977,Effectiveness of 3D virtual imaging0


In [13]:
# この値を境に、モデルの出力を 0 と 1 にします。
border = len(train[train["judgement"] == 1]) / len(train["judgement"])
print(border)

border = 0.02

0.023282372444280715


In [15]:
k_fold = 5

In [16]:
def get_train_data(train):

    #使用sklearn的StratifiedKFold类（k折，随机），
    #详情看https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
    Fold = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=seed)
    #使用split方法给train编号，用编号划分训练集和验证集
    #n代表enumerate带来的每个(train_index, val_index)的index，
    #for循环n_splits次，n也就从0到4
    for n, (train_index, val_index) in enumerate(Fold.split(train, train["judgement"])):
        #在train中新增"fold"列，用以记录"val_index"行属于第几折
        train.loc[val_index, "fold"] = int(n)
    train["fold"] = train["fold"].astype(np.uint8)
    
    #返回的训练集中列有"id","text","submission","fold"
    return train

In [17]:
def get_test_data(test):
    return test

In [18]:
train = get_train_data(train)

In [19]:
train

,id,judgement,text,fold
0,0,0,One-year age changes in MRI brain volumes in o...,1
1,1,0,Supportive CSF biomarker evidence to enhance t...,1
2,2,0,Occurrence of basal ganglia germ cell tumors w...,1
3,3,0,New developments in diagnosis and therapy of C...,2
4,4,0,Prolonged shedding of SARS-CoV-2 in an elderly...,1
...,...,...,...,...
27140,27140,0,The amyloidogenic pathway of amyloid precursor...,4
27141,27141,0,Technologic developments in radiotherapy and s...,1
27142,27142,0,Novel screening cascade identifies MKK4 as key...,0
27143,27143,0,Visualization of the gall bladder on F-18 FDOP...,2


In [20]:
train["text"].map(lambda x: len(x)).max()

9690

In [21]:
test["text"].map(lambda x: len(x)).max()

12934

In [22]:
trn_idx = train[train["fold"] != 1].index

In [24]:
class BaseDataset(Dataset):
    def __init__(self, df, model_name, include_labels=True):
        #引入BertTokenizer.from_pretrained类
        #详情参照https://huggingface.co/transformers/main_classes/tokenizer.html#transformers.PreTrainedTokenizer.batch_encode_plus
        tokenizer = T.BertTokenizer.from_pretrained(model_name)

        self.df = df
        self.include_labels = include_labels

        self.text = df["text"].tolist()
        #使用batch_encode_plus对多个句子编码
        #编码包括：
        #Tokenize sentences
        #Prepend the [CLS] token to the start 
        #Append the [SEP] token to the end
        #Map tokens to their IDs
        #Make IDs same max_length  
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',            
            max_length = 256,
            truncation = True,
            return_attention_mask=True
        )
        
        if self.include_labels:
            self.labels = df["judgement"].values

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        #为啥这里要加[idx]?
        #表示标记id表和注意力掩码的索引
        input_ids = torch.tensor(self.encoded['input_ids'][idx])
        attention_mask = torch.tensor(self.encoded['attention_mask'][idx])

        if self.include_labels:
            label = torch.tensor(self.labels[idx]).float()
            return input_ids, attention_mask, label

        return input_ids, attention_mask

In [25]:
class BaseModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()

        self.model = T.BertForSequenceClassification.from_pretrained(model_name, num_labels=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        out = self.model(input_ids=input_ids, attention_mask=attention_mask)
        out = self.sigmoid(out.logits).squeeze()

        return out

In [26]:
#辅助函数
class AverageMeter(object):
    #计算平均值用
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    #将秒转化为分
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


def timeSince(since, percent):
    #持续时间
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))

In [27]:
def train_fn(train_loader, model, criterion, optimizer, epoch, device):
    start = end = time.time()
    losses = AverageMeter()

    # switch to train mode
    model.train()

    for step, (input_ids, attention_mask, labels) in enumerate(train_loader):
        optimizer.zero_grad()

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        y_preds = model(input_ids, attention_mask)

        loss = criterion(y_preds, labels)

        # record loss
        #使用AverageMeter()更新并记录误差
        losses.update(loss.item(), batch_size)
        loss.backward()

        optimizer.step()

        if step % 100 == 0 or step == (len(train_loader) - 1):
            print(
                f"Epoch: [{epoch + 1}][{step}/{len(train_loader)}] "
                f"Elapsed {timeSince(start, float(step + 1) / len(train_loader)):s} "
                f"Loss: {losses.avg:.4f} "
            )
    
    return losses.avg

In [28]:
def valid_fn(valid_loader, model, criterion, device):
    start = end = time.time()
    losses = AverageMeter()

    # switch to evaluation mode
    model.eval()
    preds = []

    for step, (input_ids, attention_mask, labels) in enumerate(valid_loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        # compute loss
        with torch.no_grad():
            y_preds = model(input_ids, attention_mask)

        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)

        # record score
        preds.append(y_preds.to("cpu").numpy())

        if step % 100 == 0 or step == (len(valid_loader) - 1):
            print(
                f"EVAL: [{step}/{len(valid_loader)}] "
                f"Elapsed {timeSince(start, float(step + 1) / len(valid_loader)):s} "
                f"Loss: {losses.avg:.4f} "
            )

    predictions = np.concatenate(preds)
    return losses.avg, predictions

In [29]:
from transformers import AutoTokenizer, AutoModel

In [30]:
inference_dataloader_batchsize = 28

#对测试集进行推理，获得预测结果
def inference():
    predictions = []

    test_dataset = BaseDataset(test, "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext", include_labels=False)
    test_loader = DataLoader(
        test_dataset, batch_size=inference_dataloader_batchsize,
        shuffle=False, num_workers=0, pin_memory=True,
    )

    for fold in range(k_fold):
        LOGGER.info(f"========== model: microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext fold: {fold} inference ==========")
        model = BaseModel("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
        model.to(device)
        model.load_state_dict(torch.load(OUTPUT_DIR + f"PubMedBERT_base_uncased_fold{fold}_best.pth")["model"])
        model.eval()
        preds = []
        for i, (input_ids, attention_mask) in tqdm(enumerate(test_loader), total=len(test_loader)):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            with torch.no_grad():
                y_preds = model(input_ids, attention_mask)
            preds.append(y_preds.to("cpu").numpy())
        preds = np.concatenate(preds)
        predictions.append(preds)
    predictions = np.mean(predictions, axis=0)

    return predictions

In [31]:
trainloop_dataloader_batchsize = 28

#将经过get_train_data()函数处理的train放入到train_loop()中
def train_loop(train, fold):

    LOGGER.info(f"========== fold: {fold} training ==========")
    # ====================================================
    # Data Loader
    # ====================================================
    #train["fold"]不是第fold的行，其index记为trn_idx；
    #反之其index记为val_index
    trn_idx = train[train["fold"] != fold].index
    val_idx = train[train["fold"] == fold].index
    
    #对训练集验证集重新reset索引
    train_folds = train.loc[trn_idx].reset_index(drop=True)
    valid_folds = train.loc[val_idx].reset_index(drop=True)
    
    #使用BasedataSet函数对数据集进行编码
    train_dataset = BaseDataset(train_folds, "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
    valid_dataset = BaseDataset(valid_folds, "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

    train_loader = DataLoader(
        train_dataset,
        batch_size=trainloop_dataloader_batchsize,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
        drop_last=True,
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=trainloop_dataloader_batchsize,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        drop_last=False,
    )

    # ====================================================
    # Model
    # ====================================================
    model = BaseModel("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
    model.to(device)

    optimizer = T.AdamW(model.parameters(),
                        lr=2e-5,)

    criterion = nn.BCELoss()

    # ====================================================
    # Loop
    # ====================================================
    best_score = -1
    best_loss = np.inf

    for epoch in range(3):
        start_time = time.time()
        
        # train
        #train_fn函数返回训练集平均误差
        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, device)

        # eval
        #valid_fn函数返回验证集平均误差和预测结果
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
        valid_labels = valid_folds["judgement"].values

        # scoring
        score = fbeta_score(valid_labels, np.where(preds < border, 0, 1), beta=7.0)

        elapsed = time.time() - start_time
        LOGGER.info(
            f"Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s"
        )
        LOGGER.info(f"Epoch {epoch+1} - Score: {score}")
        
        #如果验证集的得分比最佳得分好，就取代最佳得分，
        #并将模型参数保存下来
        if score > best_score:
            best_score = score
            LOGGER.info(f"Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model")
            torch.save(
                {"model": model.state_dict(), "preds": preds}, OUTPUT_DIR + f"PubMedBERT_base_uncased_fold{fold}_best.pth"
            )
    
    #读取最佳模型参数
    check_point = torch.load(OUTPUT_DIR + f"PubMedBERT_base_uncased_fold{fold}_best.pth")

    valid_folds["preds"] = check_point["preds"]

    #返回包含最佳预测结果的验证集
    return valid_folds

In [32]:
#计算F7score得分
def get_result(result_df):
    preds = result_df["preds"].values
    labels = result_df["judgement"].values
    score = fbeta_score(labels, np.where(preds < border, 0, 1), beta=7.0)
    LOGGER.info(f"Score: {score:<.5f}")

In [34]:
# Training
oof_df = pd.DataFrame()
for fold in range(k_fold):
    _oof_df = train_loop(train, fold)
    oof_df = pd.concat([oof_df, _oof_df])
    LOGGER.info(f"========== fold: {fold} result ==========")
    #第k折的得分
    get_result(_oof_df)

# CV result
#k折平均得分
LOGGER.info(f"========== CV ==========")
get_result(oof_df)


========== fold: 0 training ==========
Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceCla

Epoch: [1][0/775] Elapsed 0m 1s (remain 14m 54s) Loss: 0.6198 
Epoch: [1][100/775] Elapsed 0m 53s (remain 5m 55s) Loss: 0.1419 
Epoch: [1][200/775] Elapsed 1m 45s (remain 5m 1s) Loss: 0.1226 
Epoch: [1][300/775] Elapsed 2m 38s (remain 4m 8s) Loss: 0.1034 
Epoch: [1][400/775] Elapsed 3m 30s (remain 3m 16s) Loss: 0.0942 
Epoch: [1][500/775] Elapsed 4m 23s (remain 2m 23s) Loss: 0.0873 
Epoch: [1][600/775] Elapsed 5m 15s (remain 1m 31s) Loss: 0.0815 
Epoch: [1][700/775] Elapsed 6m 8s (remain 0m 38s) Loss: 0.0768 
Epoch: [1][774/775] Elapsed 6m 47s (remain 0m 0s) Loss: 0.0742 
EVAL: [0/194] Elapsed 0m 0s (remain 0m 37s) Loss: 0.0093 
EVAL: [100/194] Elapsed 0m 17s (remain 0m 16s) Loss: 0.0386 


Epoch 1 - avg_train_loss: 0.0742  avg_val_loss: 0.0387  time: 442s
Epoch 1 - Score: 0.9049102507046434
Epoch 1 - Save Best Score: 0.9049 Model


EVAL: [193/194] Elapsed 0m 34s (remain 0m 0s) Loss: 0.0387 
Epoch: [2][0/775] Elapsed 0m 0s (remain 6m 53s) Loss: 0.0476 
Epoch: [2][100/775] Elapsed 0m 53s (remain 5m 56s) Loss: 0.0382 
Epoch: [2][200/775] Elapsed 1m 47s (remain 5m 6s) Loss: 0.0369 
Epoch: [2][300/775] Elapsed 2m 40s (remain 4m 12s) Loss: 0.0396 
Epoch: [2][400/775] Elapsed 3m 32s (remain 3m 18s) Loss: 0.0380 
Epoch: [2][500/775] Elapsed 4m 25s (remain 2m 25s) Loss: 0.0394 
Epoch: [2][600/775] Elapsed 5m 19s (remain 1m 32s) Loss: 0.0390 
Epoch: [2][700/775] Elapsed 6m 12s (remain 0m 39s) Loss: 0.0393 
Epoch: [2][774/775] Elapsed 6m 51s (remain 0m 0s) Loss: 0.0390 
EVAL: [0/194] Elapsed 0m 0s (remain 0m 36s) Loss: 0.0034 
EVAL: [100/194] Elapsed 0m 17s (remain 0m 16s) Loss: 0.0361 


Epoch 2 - avg_train_loss: 0.0390  avg_val_loss: 0.0361  time: 445s
Epoch 2 - Score: 0.9092298288508557
Epoch 2 - Save Best Score: 0.9092 Model


...

Epoch 2 - avg_train_loss: 0.0318  avg_val_loss: 0.0683  time: 444s
Epoch 2 - Score: 0.6662486283116477


EVAL: [193/194] Elapsed 0m 33s (remain 0m 0s) Loss: 0.0683 
Epoch: [3][0/775] Elapsed 0m 0s (remain 6m 39s) Loss: 0.0006 
Epoch: [3][100/775] Elapsed 0m 53s (remain 5m 58s) Loss: 0.0242 
Epoch: [3][200/775] Elapsed 1m 46s (remain 5m 5s) Loss: 0.0227 
Epoch: [3][300/775] Elapsed 2m 40s (remain 4m 12s) Loss: 0.0202 
Epoch: [3][400/775] Elapsed 3m 33s (remain 3m 18s) Loss: 0.0206 
Epoch: [3][500/775] Elapsed 4m 26s (remain 2m 25s) Loss: 0.0219 
Epoch: [3][600/775] Elapsed 5m 19s (remain 1m 32s) Loss: 0.0211 
Epoch: [3][700/775] Elapsed 6m 12s (remain 0m 39s) Loss: 0.0215 
Epoch: [3][774/775] Elapsed 6m 52s (remain 0m 0s) Loss: 0.0212 
EVAL: [0/194] Elapsed 0m 0s (remain 0m 36s) Loss: 0.0006 
EVAL: [100/194] Elapsed 0m 17s (remain 0m 16s) Loss: 0.0412 


Epoch 3 - avg_train_loss: 0.0212  avg_val_loss: 0.0457  time: 446s
Epoch 3 - Score: 0.8564814814814816


EVAL: [193/194] Elapsed 0m 33s (remain 0m 0s) Loss: 0.0457 


========== fold: 4 result ==========
Score: 0.88054
========== CV ==========
Score: 0.90757


In [35]:
# Save OOF result
# 存储包含最佳预测结果的验证集
oof_df.to_csv(OUTPUT_DIR + "oof_df_pubmed_12.csv", index=False)

In [36]:
# Inference
predictions = inference()

========== model: microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext fold: 0 inference ==========
Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initia

========== model: microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext fold: 1 inference ==========


Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Ber

========== model: microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext fold: 2 inference ==========


Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Ber

========== model: microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext fold: 3 inference ==========


Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Ber

========== model: microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext fold: 4 inference ==========


Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Ber

In [37]:
borders = []
list = range(10,90,1)
for x in list:
    x = x/1000
    borders.append(x)


for border in borders:
    preds = np.where(predictions < border, 0, 1)
    # submission
    sub["judgement"] = preds
    n = str(border)
    sub.to_csv(OUTPUT_DIR + "bert"+n+".csv", index=False, header=False)